In [1]:
from google.cloud import bigquery
import json
from collections import OrderedDict
import pprint
import pandas as pd
import numpy as np


In [2]:
from datetime import datetime
TIMESTAMP = datetime.now()

# 実験メニューをBQから読み込み

In [23]:
client = bigquery.Client()

In [4]:
target_table = "【出力先BQテーブル】"

In [5]:
def load_from_bq(query):
    return client.query(query).to_dataframe()

In [6]:
query = """
【実験メニューをBQからロード】
"""
df_menu = load_from_bq(query)

In [24]:
df_menu

,p_max_depth,p_n_estimators,metrics
0,1,2,"test_score, val_score"
1,2,2,"test_score, val_score"
2,3,2,"test_score, val_score"
3,1,3,"test_score, val_score"
4,2,3,"test_score, val_score"
5,3,3,"test_score, val_score"


In [25]:
df_params = df_menu.drop(["metrics"], axis = 1)
metrics = df_menu["metrics"][0]

In [26]:
params = df_params.to_dict(orient='records')

In [27]:
notebook_dir = "【ノートブックのurl】"
pyfile_dir = "generated_code.py"

In [28]:
arg_str_ = ""
for n, i in enumerate(params[0].keys()):
    if n < len(params[0].keys()) - 1:
        arg_str_ += i + ", "
    else:
        arg_str_ += i

In [29]:
# jsonを処理してpyファイル化する
def json_process(json_, arg_str_, metrics):
    li = []
    for i in json_["cells"]:
        if i["cell_type"] == "code":
            if i["metadata"]["tags"] != ["param"]:
                li+= i["source"]
                
    str_ = "def main("+ arg_str_ + "): \n"
    for j in li:
        str_ += "    " + j + "\n"
    
    str_ += "    " + "return " +  metrics
    return str_

In [30]:
def generate_code(notebook_dir, pyfile_dir, arg_str_, metrics):
    # notebookをjsonとして読み込み
    with open(notebook_dir) as f:
        json_ = json.load(f)
    
    # jsonの中身を処理
    str_ = json_process(json_, arg_str_, metrics)
    
    with open(pyfile_dir, "w") as f:
        f.write(str_)
    
    return json_, str_

In [32]:
json_, code = generate_code(notebook_dir, pyfile_dir, arg_str_,  metrics)

In [37]:
json_["cells"]

[{'cell_type': 'markdown',
  'id': 'b2a71eb6-0a70-45d1-ba2b-cfc87d3ba2e7',
  'metadata': {'tags': []},
  'source': ['# param ']},
 {'cell_type': 'code',
  'execution_count': 47,
  'id': 'a3259b8c-31ac-4ce0-b52c-caa45574dec2',
  'metadata': {'tags': ['param']},
  'outputs': [],
  'source': ['p_max_depth = 1\n', 'p_n_estimators = 2']},
 {'cell_type': 'markdown',
  'id': '9f6ce31f-8cd7-4b0b-b6e6-6cd751f45594',
  'metadata': {'tags': []},
  'source': ['# import']},
 {'cell_type': 'code',
  'execution_count': 48,
  'id': '524d9d21-4a0b-4a4a-803e-fbbbb67f66b5',
  'metadata': {'tags': ['import']},
  'outputs': [],
  'source': ['from google.cloud import bigquery']},
 {'cell_type': 'code',
  'execution_count': 49,
  'id': '2f0501ec-9bba-466d-88fc-0c130ac0d6ed',
  'metadata': {'tags': ['import']},
  'outputs': [],
  'source': ['import pandas as pd']},
 {'cell_type': 'code',
  'execution_count': 50,
  'id': '5ad49114-4865-48ec-b518-63e68d1f6f96',
  'metadata': {'tags': ['import']},
  'outputs': [

# 作成したpythonファイルをimport

In [14]:
from generated_code import main

In [15]:
columns = list(params[0].keys()) + metrics.replace(" ", "").split(",")
li = []
for param in params:
    i = main(**param)
    li.append(list(param.values()) + list(i))

In [18]:
result = pd.DataFrame(li, columns = columns)

In [19]:
result.insert(0, "version", "version 1.0")
result.insert(0, "timestamp", TIMESTAMP)

# BQへのupload

In [20]:
bq_coltypes = []
for col_name, dtype in zip(result.columns, result.dtypes):

        if dtype == np.object_:
            bq_coltypes.append(bigquery.SchemaField(col_name, bigquery.enums.SqlTypeNames.STRING))
            #sample_result[col_name] = sample_result[col_name].astype(str)

        elif dtype == np.int64:
            bq_coltypes.append(bigquery.SchemaField(col_name, bigquery.enums.SqlTypeNames.INTEGER))

        elif dtype == np.float64:
            bq_coltypes.append(bigquery.SchemaField(col_name, bigquery.enums.SqlTypeNames.FLOAT))

        else:
            #日付系はいっぱいあってしんどいので例外処理
            bq_coltypes.append(bigquery.SchemaField(col_name, bigquery.enums.SqlTypeNames.TIMESTAMP))

In [21]:
job_config = bigquery.LoadJobConfig(
        schema = bq_coltypes
)

In [22]:
job = client.load_table_from_dataframe(
        result, target_table, job_config = job_config
)    